In [1]:
# will autoupdate any of the packages imported:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
# import pyclifford
from pyclifford import utils, paulialg, stabilizer
from pyclifford.utils import acq, ipow, clifford_rotate, pauli_transform

In [3]:
def decompose(g, gs, ps):
    '''  Decompose a pauli string to phase*destabilizers*stabilizers
    Parameters:
    g: int(2*N) - the binary vector of a pauli string
    gs: int(2*N,2*N) - the full tableau
    ps: int(2*N) - phase of stabilizer and destabilizer
    
    Returns:
    phase: int - phase in terms of imaginery power
    b: int(N) - binary encoding of destabilizer decomposition
    c: int(N) - binary encoding of stabilizer decomposition
    '''
    phase = 0
    tmp_p = np.zeros_like(g)
    N = gs.shape[0]//2
    b = np.zeros(N).astype(int)
    c = np.zeros(N).astype(int)
    for i in range(N):
        if acq(g,gs[i]): #anti-commute
            b[i] = 1
            phase = phase - ipow(tmp_p,gs[i+N]) + ps[i+N]
            tmp_p = (tmp_p+gs[i+N])%2
    for i in range(N):
        if acq(g,gs[i+N]): #anti-commute
            c[i] = 1
            phase = phase - ipow(tmp_p,gs[i]) + ps[i]
            tmp_p = (tmp_p+gs[i])%2
    return phase%4, tmp_p, b, c

The above function will decompose a pauli string into combination of destabilizer generators and stabilizer generators.

Let's see an example, if the stabilizer generators are $g_0=-ZZ$ and $g_1=XX$ and destabilizer generators are $d_0=IX$, $d_1=XI$.

In [4]:
gs = stabilizer.stabilizer_state("-ZZ","XX").gs
ps = stabilizer.stabilizer_state("-ZZ","XX").ps

In [5]:
gs

array([[0, 1, 0, 1],
       [1, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

In [6]:
ps

array([2, 0, 0, 0])

In [7]:
decompose(np.array([1,0,1,1]),gs,ps)

(1, array([1, 0, 1, 1]), array([0, 1]), array([1, 1]))

From the result, we see pauli string $XY$ can be decomposed as
$$XY = i^1 d_0^0 d_1^1 g_0^1 g_1^1=i(XI)(-ZZ)(XX)$$

In [8]:
# def list2num(lst):
#     return np.sum(2**np.arange(len(lst)) * np.array(list(reversed(lst))))

# def num2list(num, N):
#     assert num < 2**N
#     return np.array(list(map(int, bin(num)[2:].zfill(N))))

In [9]:
class GeneralStabilizerState(object):
    '''Represents a stabilizer state. This is a subclass of PauliList.
        rho = 1/2^r prod_{a=1}^{N-r} (1+ Pauli[g_a,p_a])/2

    The stabilizer state is specified by a stablizer tableau, as a binary matrix
    of the shape (2*N, 2*N).
        rows [0,r) - standby stabilizers
        rows [r,N) - active stabilizers (g_a)
        rows [N,N+r) - standby destabilizers
        rows [N+r,2*N) - active destabilizers
    The stabilizers and destablizers in the tableau forms a list of Pauli 
    operators, which can be represented as a subclass of PauliList.

    Parameters:
    gs: int (2*N, 2*N) - strings of Pauli operators in the stabilizer tableau.
    ps: int (2*N) - phase indicators (should only be 0 or 2).
    r:  int  - number of logical qubits (log2 rank of density matrix)'''
    # def __init__(self, *args, **kwargs):
    def __init__(self, chi, gs, ps):
        self.chi = chi
        self.gs = gs
        self.ps = ps
        
    # def __repr__(self):
    #     ''' will only show active stabilizers, 
    #         to see the full stabilizer tableau, convert to PauliList by [:] '''
    #     subrepr = repr(self.stabilizers)
    #     if subrepr == '':
    #         return 'StabilizerState()'
    #     else:
    #         return 'StabilizerState(\n{})'.format(subrepr).replace('\n','\n  ')

    @property
    def stabilizers(self):
        return self[self.r:self.N]
    
    def copy(self):
        return StabilizerState(self.gs.copy(), self.ps.copy()).set_r(self.r)

    def rotate_by(self, generator, mask=None):
        # perform Clifford transformation by Pauli generator (in-place)
        if mask is None:
            clifford_rotate(generator.g, generator.p, self.gs, self.ps)
        else:
            mask2 = numpy.repeat(mask,  2)
            self.gs[:,mask2], self.ps = clifford_rotate(
                generator.g, generator.p, self.gs[:,mask2], self.ps)
        return self
        
    def transform_by(self, generator, mask=None):
        # perform Clifford transformation by Clifford map (in-place)
        if mask is None:
            self.gs, self.ps = pauli_transform(self.gs, self.ps, 
                clifford_map.gs, clifford_map.ps)
        else:
            mask2 = numpy.repeat(mask, 2)
            self.gs[:,mask2], self.ps = pauli_transform(
                self.gs[:,mask2], self.ps, clifford_map.gs, clifford_map.ps)
        return self
    
    def pauli_channel(self, phi, F):
        # perform general clifford channel
        pass
        

### Test usage of generalized stabilizer

In [26]:
# test
h1 = np.array([0, 1, 0, 0])
h2 = np.array([1, 0, 1, 0])
key = (list2num(h1), list2num(h2))
ele = 0.125
chi = {key: ele}

ss = stabilizer.stabilizer_state("ZIII","IZII","IIZI","IIIZ")
gs = ss.gs
_, N2 = gs.shape
N = N2//2
ds = gs[N:2*N]
ps = ss.ps

gss = GeneralStabilizerState(chi, gs, ps)

In [24]:
gss.chi

{(4, 10): 0.125}

In [25]:
gss.gs

array([[0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0]])

In [28]:
h1 = (0, 1, 0, 0)
h2 = (1, 0, 1, 0)
key = (h1, h2)
dic = {key: 0.125}

In [30]:
mask = [0, 1, 0, 1, 0]
mask2 = np.repeat(mask, 2)
mask2

array([0, 0, 1, 1, 0, 0, 1, 1, 0, 0])